In [ ]:
import numpy as np 
import pandas as pd
import zipfile
import glob
from PIL import Image
import copy
import random
from time import time
from typing import Any, Dict
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation

# 0. Import Data

In [ ]:
# Unzip the dataset
with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip", "r") as z:
    z.extractall(".")
# Make Data directory (limited to dog data)
!mkdir -p images/dog
!mv train/dog.*.jpg images/dog/

In [ ]:
# Make data set with resized and normalized
image_size = 64
dataset = torchvision.datasets.ImageFolder(
    root = "images", 
    transform = transforms.Compose(
        [transforms.Resize((image_size, image_size)),
         transforms.CenterCrop(image_size),
         transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

# 1. Constract DCGAN Model

Refer to https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

## 1.1. Generator

In [ ]:
# Set latent vector size
n_z = 100
# Set image size
n_f = 64
# Set　channel size
n_c = 3
class Generator(nn.Module):
    def __init__(self, n_z, n_f, n_c, relu_slope=0.2):
        super().__init__()
        
        self.network = nn.Sequential(nn.ConvTranspose2d(in_channels=n_z, out_channels=n_f*8, kernel_size=4, stride=1, padding=0, bias=False),
                                     nn.BatchNorm2d(n_f*8),
                                     nn.LeakyReLU(negative_slope=relu_slope,inplace=True),
                                     nn.ConvTranspose2d(in_channels=n_f*8, out_channels=n_f*4, kernel_size=4, stride=2, padding=1, bias=False),
                                     nn.BatchNorm2d(n_f*4),
                                     nn.LeakyReLU(negative_slope=relu_slope,inplace=True),
                                     nn.ConvTranspose2d(in_channels=n_f*4, out_channels=n_f*2, kernel_size=4, stride=2, padding=1, bias=False),
                                     nn.BatchNorm2d(n_f*2),
                                     nn.LeakyReLU(negative_slope=relu_slope,inplace=True),
                                     nn.ConvTranspose2d(in_channels=n_f*2, out_channels=n_f, kernel_size=4, stride=2, padding=1, bias=False),
                                     nn.BatchNorm2d(n_f),
                                     nn.LeakyReLU(negative_slope=relu_slope,inplace=True),
                                     nn.ConvTranspose2d(in_channels=n_f, out_channels=n_c, kernel_size=4, stride=2, padding=1, bias=False),
                                     nn.Tanh()
                                    )
    def forward(self, x):
        return self.network(x)

## 1.2. Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, n_f, n_c, relu_slope=0.2):
        super().__init__()
        self.network = nn.Sequential(nn.Conv2d(in_channels=n_c, out_channels=n_f, kernel_size=4, stride=2, padding=1, bias=False),
                                     nn.LeakyReLU(negative_slope=relu_slope,inplace=True),
                                     nn.Conv2d(in_channels=n_f, out_channels=n_f*2, kernel_size=4, stride=2, padding=1, bias=False),
                                     nn.BatchNorm2d(n_f*2),
                                     nn.LeakyReLU(negative_slope=relu_slope,inplace=True),
                                     nn.Conv2d(in_channels=n_f*2, out_channels=n_f*4, kernel_size=4, stride=2, padding=1, bias=False),
                                     nn.BatchNorm2d(n_f*4),
                                     nn.LeakyReLU(negative_slope=relu_slope,inplace=True),
                                     nn.Conv2d(in_channels=n_f*4, out_channels=n_f*8, kernel_size=4, stride=2, padding=1, bias=False),
                                     nn.BatchNorm2d(n_f*8),
                                     nn.LeakyReLU(negative_slope=relu_slope,inplace=True),
                                     nn.Conv2d(in_channels=n_f*8, out_channels=1, kernel_size=4, stride=1, padding=0, bias=False),
                                     nn.Sigmoid()
                                    )
    def forward(self, x):
        return self.network(x)

In [ ]:
# Print model outline
gen = Generator(n_z, n_f, n_c)
print('Generator================')
print(gen)
dis = Discriminator(n_f, n_c)
print('Discriminator================')
print(dis)

## 1.3. Constract weights initialization func

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# 2. Learning

In [ ]:
# Lock random seed============================
torch.manual_seed(20220215)
torch.cuda.manual_seed(20220215)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

# Allocate device to use GPU============================
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Create models============================
dis = Discriminator(n_f, n_c).to(device)
gen = Generator(n_z, n_f, n_c).to(device)
# Initialize weights============================
dis.apply(weights_init)
gen.apply(weights_init)

# Create latent vectors used to visualize
fixed_noise = torch.randn(image_size, n_z, 1, 1, device=device)

## 2.1. Create mini-batch data

In [ ]:
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

## 2.2. Iteration

In [ ]:
# Learning rate=================
lr = 0.0001

# Set loss func=================
criterion = nn.BCELoss()

# Set optimizer=================
optimizer_d = optim.Adam(dis.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_g = optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))

# Set epochs=================
num_epochs = 20

img_list = []
g_loss = []
d_loss = []
iters = 0

## Iteration=================
for epoch in range(num_epochs):
    for i, batch in enumerate(dataloader):
        
        # (1) Update Discriminator===============================================
        dis.train()
        gen.train()
        X, _ = batch
        X = X.to(device)
        dis.zero_grad()
        # create labels (all 1) for Discriminator
        label = torch.full((X.shape[0],),1.,dtype=torch.float,device=device)
        # Predictions for real images by D (i.e., [0, 1])
        output = dis(X).view(-1)
        # Calculate loss
        errD_real = criterion(output, label)
        # Calculate gradients for Discriminator
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(batch_size, n_z, 1, 1, device=device)
        # Generate fake image batch with G
        fake = gen(noise)
        label.fill_(0.)
        # Classify all fake batch with Discriminator
        output = dis(fake.detach()).view(-1)
        # Calculate Discriminator's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizer_d.step()

        # (2) Update Generator===============================================
        gen.zero_grad()
        label.fill_(1)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = dis(fake).view(-1)
        # Calculate Generator's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for Generator
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizer_g.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        g_loss.append(errG.item())
        d_loss.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        dis.eval()
        gen.eval()

    # Save generated images for each epoch
    with torch.no_grad():
        fake = gen(fixed_noise).detach().cpu()
    img_list.append(torchvision.utils.make_grid(fake, padding=2, normalize=True))

# 3. Result

## 3.1. Visualize Training loss

In [ ]:
# Visulize loss
fig, axes = plt.subplots(1,1,figsize=(10,6))
axes.plot(d_loss,label="Discriminator",color='green')
axes.plot(g_loss,label="Generator",color='orange')
axes.set_title('Loss curve',size=16)
axes.set_xlabel('Iterations',size=16)
axes.set_ylabel('Loss',size=16)

## 3.2. Visualize generated images

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())